In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers

# FCN (based VGG16)


1. VGG16의 FC layer를 convolutionalization을 통해 1x1 convolution layer로 변환

2. 구성된 VGG16과 Up-sampling layer를 이어붙혀 FCN32 구현

3. 전체 가중치를 load하여, 계층에 대해 가중치 적용

In [2]:
based_vgg = tf.keras.applications.VGG16(include_top=True, weights='imagenet', input_shape=(224, 224, 3))

based_vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

# Extract layers

In [8]:
new_model = []

for l in based_vgg.layers[:-1]:
    new_model.append(l)
    
new_model.pop(-3)

In [9]:
new_model

# get weights, bias

In [48]:
save_weights = []

for layer in new_model[1:]:
    if layer.trainable_weights:
        weight, bias = layer.get_weights()[0], layer.get_weights()[1]
        if len(weight.shape) != 4:
            if weight.shape[0] == 25088:
                weight = np.reshape(layer.get_weights()[0], (7, 7, 512, 4096))
            else:
                weight = np.reshape(layer.get_weights()[0], (1, 1, 4096, 4096))
    else:
        continue
            
    print(layer, weight.shape, bias.shape)
    save_weights.append([weight, bias])

<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002547B526908> (3, 3, 3, 64) (64,)
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002547B583860> (3, 3, 64, 64) (64,)
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002547B5C1908> (3, 3, 64, 128) (128,)
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002547B5DE780> (3, 3, 128, 128) (128,)
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002547B5E93C8> (3, 3, 128, 256) (256,)
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002547B5E9F98> (3, 3, 256, 256) (256,)
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002547F0B0668> (3, 3, 256, 256) (256,)
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002547F0B7A20> (3, 3, 256, 512) (512,)
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002547F0C1898> (3, 3, 512, 512) (512,)
<tensorflow.python.keras.layers.convolutional

In [49]:
len(save_weights)

15

# Define based layers

In [41]:
def define_network(version):
    if version == 'fcn32':
        input_image = layers.Input(shape=(224, 224, 3))

        # Block 1
        out = layers.Conv2D(64, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(input_image)
        out = layers.Conv2D(64, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(out)

        # Block 2
        out = layers.Conv2D(128, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(128, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(out)

        # Block 3
        out = layers.Conv2D(256, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(256, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(256, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(out)

        # Block 4
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(out)

        # Block 5
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(out)

        # convolutionalization layers
        out = layers.Conv2D(4096, 7, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Dropout(0.5)(out)
        out = layers.Conv2D(4096, 1, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Dropout(0.5)(out)
        
        # final output
        out = layers.Conv2D(21, 1, 1, kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        final = layers.Conv2DTranspose(21, 64, 32, padding='same')(out) # Up sampling layers
        
        model = tf.keras.models.Model(inputs=input_image, outputs=final)

    elif version == 'fcn16':
        input_image = layers.Input(shape=(224, 224, 3))

        # Block 1
        out = layers.Conv2D(64, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(input_image)
        out = layers.Conv2D(64, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(out)

        # Block 2
        out = layers.Conv2D(128, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(128, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(out)

        # Block 3
        out = layers.Conv2D(256, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(256, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(256, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(out)

        # Block 4
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        b4_pool = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(out)

        # Block 5
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(b4_pool)
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(out)

        # convolutionalization layers
        out = layers.Conv2D(4096, 7, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Dropout(0.5)(out)
        out = layers.Conv2D(4096, 1, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Dropout(0.5)(out)
        
        # transpose layer
        out = layers.Conv2D(21, 1, 1, kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2DTranspose(21, 4, 2, padding='same')(out)
        pool_out = layers.Conv2D(21, 1, 1)(b4_pool)
        add_out = layers.Add()([out, pool_out])
        
        # final output
        final = layers.Conv2DTranspose(21, 32, 16, padding='same')(out)
        
        model = tf.keras.models.Model(inputs=input_image, outputs=final)
        
    elif version == 'fcn8':
        input_image = layers.Input(shape=(224, 224, 3))

        # Block 1
        out = layers.Conv2D(64, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(input_image)
        out = layers.Conv2D(64, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(out)

        # Block 2
        out = layers.Conv2D(128, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(128, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(out)

        # Block 3
        out = layers.Conv2D(256, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(256, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(256, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        b3_pool = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(out)

        # Block 4
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(b3_pool)
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        b4_pool = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(out)

        # Block 5
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(b4_pool)
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2D(512, 3, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(out)

        # convolutionalization layers
        out = layers.Conv2D(4096, 7, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Dropout(0.5)(out)
        out = layers.Conv2D(4096, 1, 1, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Dropout(0.5)(out)
        
        # transpose layer 1 
        out = layers.Conv2D(21, 1, 1, kernel_regularizer=tf.keras.regularizers.L2(0.0005))(out)
        out = layers.Conv2DTranspose(21, 4, 2, padding='same')(out)
        pool_out = layers.Conv2D(21, 1, 1)(b4_pool)
        add_out = layers.Add()([out, pool_out])
        
        # transpose layer 2
        out = layers.Conv2DTranspose(21, 4, 2, padding='same')(add_out)
        pool_out = layers.Conv2D(21, 1, 1)(b3_pool)
        add_out = layers.Add()([out, pool_out])
        
        # final output
        final = layers.Conv2DTranspose(21, 16, 8, padding='same')(add_out)
        
        model = tf.keras.models.Model(inputs=input_image, outputs=final)
        
    return model

In [50]:
model = define_network('fcn32')

In [51]:
model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_101 (Conv2D)          (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_102 (Conv2D)          (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_103 (Conv2D)          (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_104 (Conv2D)          (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 56, 56, 128)     

# Set weights on FCN32

In [56]:
num = 0

for i in model.layers[1:-2]:
    if i.trainable_weights:
        i.set_weights([save_weights[num][0], save_weights[num][1]])
        num += 1
    else:
        continue

In [57]:
model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_101 (Conv2D)          (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_102 (Conv2D)          (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_103 (Conv2D)          (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_104 (Conv2D)          (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 56, 56, 128)     